<a href="https://colab.research.google.com/github/weiyujust/Travelplanner/blob/main/travel_planner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Enviornment Setting

In [ ]:
!pip install flask
!pip install flask-ngrok

Gives webname Import the file through github

In [ ]:
!nohup flask run --host=0.0.0.0 --port=5000 &

!git clone https://github.com/weiyujust/Travelplanner.git

In [ ]:

import shutil
import os
os.chdir('/content')
os.getcwd()

# Define the source path
source_folder = 'Travelplanner'  # Replace 'folder1' with the actual folder name if different

# Move the contents of the first folder to the current directory
for item in os.listdir(source_folder):
    shutil.move(os.path.join(source_folder, item), '.')


In [ ]:
import time
from openai import OpenAI
from google.colab import userdata

In [ ]:
class AI_Setting:
  client = OpenAI(api_key = "sk-3jHhmvXAC4FB49QSdPdw42k6pnyCCsLj6RgOOKd1bES0DQN0", base_url = "https://api.deepbricks.ai/v1")
  default_advisor = {"role": "system", "content": """You are a travel advisor which generate travel plan for the user.
  Also, suggest the transport and routes to the location from the previous place. e.g. take East Rail line from Kowloon Tong to Central, C exit, walk 10 minutes to Lankwai Fong
  Include your accommodation suggestions at the end of the plan of each day. It can be the same or different for each day.
  Please output your plan in the following format:
  Day{Integer}:
  Location{Integer}: {text}
  Suggested route: {text}
  Description: {text}
  (Continue to add location)
  Accomodation suggestion: {text}
  Description: {text}
  (After the plans of all days are generated)
  Detailed breakdown of cost:{text} -- ${integer}
  Total estimated cost: ${integer}"""}
  default_analyzer = {"role": "system", "content": """You are a travel advisor which generates travel plans align with the users' preferences. Now you're given several plans made by the users. Please use A FEW ADJECTIVES to describe the users' preferences based on all plans given.
e.g. Theme-park lovers, low-budget"""}
  model = "gpt-4o"
  max_tokens = 2000 # MAX: 4000
  temperature = 0.5 # MAX: 2.0
  presence_penalty= 0.0
  frequency_penalty= 0.0
  model_list = ["gpt-3.5-turbo", "gpt-4o"]
  def __init__(self):
    pass

  def __del__(self):
    pass

  def print_setting(self):
    """
    Function: print the setting of the AI
    """
    print("The current setting of the AI travel advisor is as follow:")
    print("Context: You are a travel advisor which generate travel plan for the user.")
    print(f"Model: {self.model}")
    print(f"Maximum tokens: {self.max_tokens}")
    print(f"Temperature: {self.temperature}")
    print(f"Presence penalty: {self.presence_penalty}")
    print(f"Frequency penalty: {self.frequency_penalty}")
    print()

  def input_check(self, input:int):
    """
    Function: return true if the input is -1, otherwise return false
    """
    if input == -1 or input == -1.0:
      return True
    else:
      return False

  def change_setting(self):
    """
    Function: update the setting of AI
    """
    self.print_setting()
    print("Please enter the new setting below (enter -1 if there is no changes).")
    update = 0
    choice = -2
    print("Please enter the model choice: ")
    for i in range(0, len(self.model_list)):
      print(f"{i} -- {self.model_list[i]}")
    while choice < 0 or choice > len(self.model_list) or self.input_check(choice):
      try:
        choice = int(input("Model choice: "))
      except:
        print("Invalid input! Please enter again:(")
      if self.input_check(choice):
        break
      elif choice < 0 or choice > 2:
        print("Invalid value! Please enter again:(")
    if not (self.input_check(choice)):
      self.model = self.model_list[choice-1]
      update += 1

    while choice < 1500 or choice > 4000 or self.input_check(choice):
      try:
        choice = int(input("Please enter the new maximum tokens (range: 1500 - 4000): "))
      except:
        print("Invalid input! Please enter again:(")
      if self.input_check(choice):
        break
      elif choice < 1500 or choice > 4000:
        print("Invalid value! Please enter again:(")
    if not (self.input_check(choice)):
      self.max_tokens = int(choice)
      update += 1

    while ((type(choice) != float or type(choice) != int) and (choice < 0 or choice > 2)) or self.input_check(choice):
      try:
        choice = float(input("Please enter the new temperature (range: 0 - 2): "))
      except:
        print("Invalid input! Please enter again:(")
      if self.input_check(choice):
        break
      elif choice < 0 or choice > 2:
        print("Invalid value! Please enter again:(")
    if not (self.input_check(choice)):
      self.temperature = choice
      update += 1

    if update > 0:
      print("The setting updates successfully.")
      self.print_setting()
    else:
      print("Return...\n")

In [ ]:
class TravelPlanner:
  start = "Hong Kong"
  end = "Tokyo"
  duration = 3
  total_no_of_people = 3
  adult = 2
  children = 1
  infant = 0
  expected_budget = "between $5000 to $10000"
  travel_preference = "by train"
  travel_style_prompt = "Tokyo Disneyland"
  def __init__(self, start:str, end:str, duration:int, adult:int, children:int, infant:int, expected_budget:str, travel_preference:str, travel_style_prompt:str):
    """
    Function: store the input from users
    """
    self.start = start
    self.end = end
    self.duration = duration
    self.total_no_of_people = int(adult) + int(children) + int(infant)
    self.adult = adult
    self.children = children
    self.infant = infant
    self.expected_budget = expected_budget
    self.travel_preference = travel_preference
    self.travel_style_prompt = travel_style_prompt

  def __del__(self):
    """
    Function: delete the record
    """
    return 1

  def generate_plan(self, ai:AI_Setting, preference:str = ""):
    """
    Function: generate a travel plan for the users and automatically save the generated plan
    """
    self.generate_time = time.ctime() # record generated time

    command = f"Please generate a {self.duration}-days travel plan for {self.adult} adult, {self.children} children and {self.infant} infant from {self.start} to {self.end}, with expected budget {self.expected_budget}. Prefered transport: {self.travel_preference}"

    if self.travel_style_prompt != "no":
      command += f"I would like to include {self.travel_style_prompt} in my trip."

    if preference != "":
      command += f"\nMy preference: {preference}"

    chat_completion = ai.client.chat.completions.create(
    messages=[ai.default_advisor,
    {
    "role": "assistant",
    "content": command
    }
    ],
    model = ai.model,
    max_tokens = ai.max_tokens,
    temperature = ai.temperature,
    presence_penalty= ai.presence_penalty,
    frequency_penalty= ai.frequency_penalty
    )

    self.generated_record = chat_completion.choices[0].message.content

    print("Plan from AI:")
    print(chat_completion.choices[0].message.content)

    return chat_completion.choices[0].message.content


In [ ]:
class System(TravelPlanner):
  past_record = []
  preference = ""
  def __init__(self, start:str, end:str, duration:int, adult:int, children:int, infant:int, expected_budget:str, travel_preference:str, travel_style_prompt:str):
    """
    Function: set up the system
    """
    super().__init__(start, end, duration, adult, children, infant, expected_budget, travel_preference, travel_style_prompt)
    print("""Welcome to Travel Planner!
Enter:
0 -- See the example plan
1 -- Generate a customized travel plan
2 -- View all previous generated plan
3 -- Find specific generated plan
4 -- Delete previous generated plan
5 -- Clear all generated record
6 -- Show users' preference
-1 -- Quit the app

9 -- AI Setting""")
    print()

  def __del__(self):
    """
    Function: shut down the current operations
    """
    print("Thank you for using our product! Stay tuned for the new iteration:)")

  def add_record(self, plan:TravelPlanner):
    """
    Function: save all previous record
    """
    self.past_record.append(plan)

  def del_record(self):
    """
    Function: delete unwanted data
    """
    number = -2
    if len(self.past_record) == 0:
      print("No record found")
    else:
      while number < -1 or number == 0 or number > len(self.past_record):
        try:
          number = int(input("Which record do you want to delete? Please enter an integer (enter -1 to quit this page): "))
        except ValueError:
          print("Invalid input! Please enter again:(")
          number = -1
          continue

        if number < -1 or number == 0 or number > len(self.past_record):
          print("Invalid index! Please enter again:(")
        elif number == -1:
          break
      if number != -1:
        del_data = self.past_record[number-1]
        if del_data.__del__() == 1:
          print("The record is deleted sucessfully.")

  def clear_record(self):
    """
    Function: clear all records
    """
    if len(self.past_record) == 0:
      print("No record found.")
    else:
      for i in range(0, len(self.past_record)):
        del_data = self.past_record[i]
        del_data.__del__()
      self.past_record = []
      print("All records are deleted.")

  def read_all_record(self):
    """
    Function: print all record
    """
    if len(self.past_record) == 0:
      print("No record found.")
    else:
      for i in range(0, len(self.past_record)):
        print(f"Record {i+1}:")
        print(f"Generated_time: {self.past_record[i].generate_time}")
        print("Plan information: ")
        print(f"From: {self.past_record[i].start}\tTo: {self.past_record[i].end}\tDuration: {self.past_record[i].duration}")
        print(f"Number of people: {self.past_record[i].total_no_of_people}\nAdult: {self.past_record[i].adult}\tChildren: {self.past_record[i].children}\tInfant: {self.past_record[i].infant}")
        print(f"Expected budget: {self.past_record[i].expected_budget}")
        print(f"Travel Preference: {self.past_record[i].travel_preference}")
        print(f"Travel style prompt: {self.past_record[i].travel_style_prompt}")
        print(f"Generated Plan: {self.past_record[i].generated_record}")

  def read_specific_record(self):
    """
    Function: search for specific record
    """
    if len(self.past_record) == 0:
      print("No record found.")
    else:
      print("Here are the brief profile of each generated plan:")
      for i in range(0, len(self.past_record)):
        print(f"Record {i+1}:")
        print(f"Generated_time: {self.past_record[i].generate_time}")
        print("Plan information: ")
        print(f"From: {self.past_record[i].start}\tTo: {self.past_record[i].end}\tDuration: {self.past_record[i].duration}")
        print(f"Number of people: {self.past_record[i].total_no_of_people}\nAdult: {self.past_record[i].adult}\tChildren: {self.past_record[i].children}\tInfant: {self.past_record[i].infant}")
        print(f"Expected budget: {self.past_record[i].expected_budget}")
        print(f"Travel Preference: {self.past_record[i].travel_preference}")
        print(f"Travel style prompt: {self.past_record[i].travel_style_prompt}")
        print()

      choice = -2
      while choice < -1 or choice == 0 or choice > len(self.past_record):
        try:
          choice = int(input("Which record are you looking for? Please enter an integer (enter -1 to quit this page): "))
        except ValueError:
          print("Invalid input! Please enter again:(")
          choice = -1
          continue

        if choice < -1 or choice == 0 or choice > len(self.past_record):
          print("Invalid index! Please enter again:(")
        elif choice == -1:
          break
      if choice != -1:
        print("Here is the detailed information of this record:")
        print(f"Generated_time: {self.past_record[choice-1].generate_time}")
        print("Plan information: ")
        print(f"From: {self.past_record[choice-1].start}\tTo: {self.past_record[choice-1].end}\tDuration: {self.past_record[choice-1].duration}")
        print(f"Number of people: {self.past_record[choice-1].total_no_of_people}\nAdult: {self.past_record[choice-1].adult}\tChildren: {self.past_record[choice-1].children}\tInfant: {self.past_record[choice-1].infant}")
        print(f"Expected budget: {self.past_record[choice-1].expected_budget}")
        print(f"Travel Preference: {self.past_record[choice-1].travel_preference}")
        print(f"Travel style prompt: {self.past_record[choice-1].travel_style_prompt}")
        print(f"Generated Plan: {self.past_record[choice-1].generated_record}")

  def example(self, ai:AI_Setting):
    """
    Function: generate a example plan to demonstate the function of our app
    """
    print("Here is an example:")
    print("Plan information: ")
    print(f"From: {TravelPlanner.start}\tTo: {TravelPlanner.end}\tDuration: {TravelPlanner.duration}")
    print(f"Number of people: {TravelPlanner.total_no_of_people}\nAdult: {TravelPlanner.adult}\tChildren: {TravelPlanner.children}\tInfant: {TravelPlanner.infant}")
    print(f"Expected budget: {TravelPlanner.expected_budget}")
    print(f"Travel Preference: {TravelPlanner.travel_preference}")
    print(f"Travel style prompt: {TravelPlanner.travel_style_prompt}")
    print("Plan from AI: \n")
    super().generate_plan(ai)

  def analyze_preference(self, ai:AI_Setting):
    """
    Function: analyze the users' preference based on the plan generated before
    """

    command = ""
    if len(self.past_record) != 0:
      if self.preference != "":
        command += f"Analysis from the previous plans: {self.preference}\n"
      for i in range(0, len(self.past_record)):
        command += f"Day {i+1}:\n{self.past_record[i].generated_record}"
        command += "\n"
      chat_completion = ai.client.chat.completions.create(
      messages=[ai.default_analyzer,
      {
      "role": "assistant",
      "content": command
      }
      ],
      model = ai.model,
      max_tokens = ai.max_tokens,
      temperature = ai.temperature,
      presence_penalty= ai.presence_penalty,
      frequency_penalty= ai.frequency_penalty
      )
      self.preference = chat_completion.choices[0].message.content


  def print_preference(self):
    """
    Function: print user's travel preference
    """
    if self.preference != "":
      print(f"After analyzing the generated record, user preference: {self.preference}")
    else:
      print("No record found.")


Run the text version

In [ ]:
sys = System("Hong Kong", "Tokyo", 3, 2, 1, 0, "between $5000 to $10000", "by train", "Tokyo Disneyland")
ai_model = AI_Setting()

sys.example(ai_model)
print()
choice = -2

while choice != -1:
  try:
    choice = int(input("Please enter your choice:"))
  except:
    print("Invalid input! Please enter again:(")
    print()
    continue

  if choice == 0:
    sys.example(ai_model)
  elif choice == 1:
    start = input("Your departure place(e.g. Hong Kong): ")
    end = input("Your destination(e.g. Tokyo): ")
    duration = int(input("Duration of your trip(please enter an integer): "))
    adult = int(input("Number of adult(please enter an integer): "))
    children = int(input("Number of children(please enter an integer): "))
    infant = int(input("Number of infant(please enter an integer): "))
    expected_budget = 0
    while expected_budget == 0:
      try:
        budget_choice = int(input("""Expected budget(1 -- <$5000, 2 -- $5000-$10000, 3 -- $10000-$25000, 4 -- $25000-$50000, 5 -- >$50000): """))
      except:
        print("Invalid input! Please enter again:(")
        continue
      if budget_choice == 1:
        expected_budget = "less than $5000"
      elif budget_choice == 2:
        expected_budget = "from $5000 to $10000"
      elif budget_choice == 3:
        expected_budget = "from $10000 to $25000"
      elif budget_choice == 4:
        expected_budget = "from $25000 to $50000"
      elif budget_choice == 5:
        expected_budget = "more than $50000"
      else:
        print("Invalid choice! Please enter again:(")
    travel_preference = 0
    while travel_preference == 0:
      try:
        travel = int(input("""Travel preference(1 -- Train, 2 -- Bus, 3 -- Taxi, 4 -- Tram, 5 -- Ship, 6 -- On foot): """))
      except:
        print("Invalid input! Please enter again:(")
        continue
      if travel == 1:
        travel_preference = "by train"
      elif travel == 2:
        travel_preference = "by bus"
      elif travel == 3:
        travel_preference = "by taxi"
      elif travel == 4:
        travel_preference = "by tram"
      elif travel == 5:
        travel_preference = "by ship"
      elif travel == 6:
        travel_preference = "on foot"
      else:
        print("Invalid choice! Please enter again:(")
    travel_style_prompt = input("Travel style prompt(please enter locations or attractions, e.g. Hong Kong Disneyland. If you don't have any preference, please type no): ")
    travel_style_prompt.lower()
    plan = TravelPlanner(start, end, duration, adult, children, infant, expected_budget, travel_preference, travel_style_prompt)
    plan.generate_plan(ai_model, sys.preference)
    sys.add_record(plan)
    sys.analyze_preference(ai_model)
  elif choice == 2:
    sys.read_all_record()
  elif choice == 3:
    sys.read_specific_record()
  elif choice == 4:
    sys.del_record()
  elif choice == 5:
    sys.clear_record()
  elif choice == 6:
    sys.print_preference()
  elif choice == 9:
    print("Enter 1 -- View setting\nEnter 2 -- Modify setting\nEnter -1 -- Quit setting\n\n")
    action = 0
    while action != -1:
      try:
        action = int(input("Your choice: "))
      except:
        print("Invalid input! Please enter again:(")
        action = 0
        continue
      if action == 1:
        ai_model.print_setting()
      elif action == 2:
        ai_model.change_setting()
      elif action == -1:
        print("Return to the main page...")
        break
      else:
        print("Invalid choice! Please enter again:(")
  elif choice == -1:
    break;
  else:
    print("Invalid choice! Please enter again:(")
  print()

Get the link

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))


https://9msaolcug8s-496ff2e9c6d22116-5000-colab.googleusercontent.com/


Run the server

In [ ]:
from flask import Flask, request, jsonify, render_template, redirect, url_for

app = Flask(__name__)

@app.route('/')
def index():
    return render_template('TravelPlanner.html')

@app.route('/submit', methods=['POST'])
def submit():
    mylist=[]
    location = "Hong Kong"
    destination = request.form['location']
    duration = request.form['duration']
    no_of_people = request.form['no']
    budget = request.form['budget']
    Transportation_preference= request.form['tra_p']
    Trip_preference= request.form['sty_p']
    mylist.append(location)
    mylist.append(destination)
    mylist.append(duration)
    mylist.append(no_of_people)
    mylist.append(budget)
    mylist.append(Transportation_preference)
    mylist.append(Trip_preference)
    sys = System(location, destination, duration, no_of_people, 0, 0, budget, Transportation_preference, Trip_preference)
    plan = TravelPlanner(location, destination, duration, no_of_people, 0, 0, budget, Transportation_preference, Trip_preference)
    plan.generate_plan(AI_Setting(), sys.preference)
    return plan

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

